最終的な推論値を出す際に、重みでルールの切り捨てを行う

In [185]:
#加工前データ

import xlrd
import pprint
import openpyxl
import numpy as np
from datetime import datetime as dt
from sklearn.preprocessing import MinMaxScaler

start_day = dt(year=2020, month=7, day=30, hour=0)
end_day = dt(year=2021, month=1, day=11, hour=0)
test_start = dt(2020,10,1,0)
#test_end = dt(2021,1,11,0)
day_num = (end_day-start_day).days
total_data_num = day_num+1
test_data_num = (end_day-test_start).days+1
teach_data_num = total_data_num-test_data_num
print(test_data_num)
print(teach_data_num)

data_filename = 'osaka_data_before.xlsx'
output_filename = '2.xlsx'
save_filename = 'test.xlsx'

func_num = 3
x_dimension = 6
total_dimension = func_num**x_dimension
max_iter = 500
l_rate = 0.5
omega_iter = 500

theata = 10

103
63


In [186]:
#データ関連関数
def get_value_list(t_2d):
    return([[cell.value for cell in row] for row in t_2d])

def get_value(t_2d):
    return([cell.value for cell in row] for row in t_2d)

def get_list_2d(sheet, start_row, end_row, start_col, end_col):
    return get_value_list(sheet.iter_rows(min_row=start_row,
                                          max_row=end_row,
                                          min_col=start_col,
                                         max_col=end_col))
def get_list_2d_T(sheet, start_row, end_row, start_col, end_col):
    return get_value_list(sheet.iter_cols(min_row=start_row,
                                          max_row=end_row,
                                          min_col=start_col,
                                         max_col=end_col))

def write_list_2d(sheet, l_2d, start_row, start_col):
    for y, row in enumerate(l_2d):
        for x, cell in enumerate(row):
            sheet.cell(row=start_row + y,
                       column=start_col + x,
                       value=l_2d[y][x])

In [187]:
#データを扱いやすく変形
def set_data(y, Y, data, b, c, omega):
    for i in range(total_data_num):
        temp =[]
        temp.append((Y[i][0]-np.min(Y))/(np.max(Y)-np.min(Y)))
        for i in range(theata):
            temp.append((np.mean(Y)-np.min(Y))/(np.max(Y)-np.min(Y)))
        y.append(temp)
        
    """for i in range(total_dimension):
        omega[i] = (i+1)/total_dimension"""

#データを扱いやすく変形
def set_data(y, Y, data, b, c, omega):
    
    for i in range(total_data_num):
        temp =[]
        temp.append(Y[i][0])
        for i in range(theata):
            temp.append(np.mean(Y))
        y.append(temp)
        
    '''for i in range(total_dimension):
        omega[i] = (i+1)/total_dimension'''

In [188]:
#最急降下法 改良
def gradient_descent(total_m, m, y, w, omega, mse, threshold):
    #w = np.ones(total_dimension)
    #temp_w = 0
    
    """#後件部の学習
    for i in range(omega_iter):    
        for j in range(teach_data_num):
            for dim in range(total_dimension):
                omega[dim] -= l_rate * (-y[j][0]+y[j][1]) * m[j][dim] / total_m[j]
                w[dim] = w[dim] - l_rate *  (-y[j][0]+y[j][1]) * m[j][dim] * (omega[dim]-y[j][1]) / total_m[j]
                '''if temp_w < 0:
                    w[dim] = 0
                else:
                    w[dim] = temp_w'''
            temp1 = 0
            temp2 = 0
            for r, W, ome in zip(m[j], w, omega):
                temp1 += r * W * ome
                temp2 += r * W
            total_m[j] = temp2
            y[j][1] = temp1/total_m[j]
        #print(y[0][1])
        MES(y, mse, i, w, omega, 'teach')
    #重みの学習結果を残しておく
    #時間短縮のため
    save_result(y, w, omega)
    
    print("後件部学習終了")
    print("重み学習終了")"""
    
    #testデータで
    for j, n in enumerate(threshold):
        print(j,n)
        for i in range(teach_data_num,total_data_num):
            total_temp = 0
            temp = 0
            for num, weight in enumerate(w):
                if weight > n:
                    """if j == 7:
                        print(weight)"""
                    total_temp += m[i][int(num)]*weight
                    temp += m[i][int(num)]*weight*omega[int(num)]
                    """for mu,ome in zip(m[i],omega):
                        total_temp += mu*weight
                        temp += mu*ome*weight """

            #エラー処理
            if total_temp == 0:
                print("Error total_m is 0")
                y[i][int(j+1)] = 1.0
            else: 
                y[i][int(j+1)] = temp/total_temp

        """    temp = 0
            for r, W, ome in zip(m[i], w, omega):
                if W > n:
                    temp += r * W * ome"""
            
            

    MES(y, mse, theata, w, omega, 'test')
    save_result(y, w, omega)

In [189]:
def MES(y, mse, n, w, omega, litera):
    temp = 0
    
    if litera=='teach':
        for i in range(teach_data_num):
            temp += (y[i][1] - y[i][0])**2
        mse = temp/teach_data_num
    else:
        for j in range(n):
            temp = 0
            for i in range(teach_data_num,total_data_num):
                temp += (y[i][j+1] - y[i][0])**2
            mse[j] = temp/test_data_num
    
    wc = openpyxl.load_workbook('./output/osaka/'+save_filename, data_only=True)
    for sheet_name in wc.sheetnames:
        if sheet_name == 'output':
            for i in range(n):
                wc["sq_error"].cell(column=1, row=1+i, value = mse[i])
            
            wc.save('./output/osaka/'+save_filename) 
        """else:
            err_sheet = wc.create_sheet('sq_error')
            err_sheet.cell(column=1, row=1+n, value = mse)"""
    return 

In [190]:
#excelの保存
def save_result(output_list, w, omega):
    
    wc = openpyxl.load_workbook('./output/osaka/'+save_filename, data_only=True)
    for sheet_name in wc.sheetnames:
        if sheet_name == 'output':
            write_list_2d(wc["output"],[['理想値','出力値']], 1, 1)
            write_list_2d(wc["output"],output_list, 2, 1)
            
            

            #wc["sq_error"].cell(column=1, row=1, value = mse)
            
            omega_sheet = wc["omega"]
            for i in range(total_dimension):
                omega_sheet.cell(column=1, row=i+1, value = omega[i])
                
            w_sheet = wc["w"]
            for i in range(total_dimension):
                w_sheet.cell(column=1, row=i+1, value = w[i])
            
            wc.save('./output/osaka/'+save_filename)
            print("Already exist. Save complited ")  
            return 
        
    output_sheet = wc.create_sheet('output')
    write_list_2d(output_sheet,[['理想値','出力値']], 1, 1)
    write_list_2d(output_sheet,output_list, 2, 1)
    #err_sheet = wc.create_sheet('sq_error')
    #err_sheet.cell(column=1, row=1, value = mse)
    
    w_sheet = wc.create_sheet('w')
    for i in range(total_dimension):
        w_sheet.cell(column=1, row=i+1, value = w[i])
        
    w_sheet = wc.create_sheet('omega')
    for i in range(total_dimension):
        w_sheet.cell(column=1, row=i+1, value = omega[i])
    
    wc.save('./output/osaka/'+save_filename)
    print("Save complited ")
    return 

In [191]:
#出力yを導出する関数
def make_output(total_m, m, y, fun_data, data, omega, c, b, w, sq_err, threshold):
    #rule(m, total_m, fun_data, data, c, b, w)
    
    print("学習開始")
    #学習
    gradient_descent(total_m, m, y, w, omega, sq_err, threshold)
    
    print("学習完了")
    
    return 0

In [194]:
def main():
    '''
    data：データ
    y：病床占有率
    fun_data：関数を通したデータの値
    m：ルールに対する適合度
    total_m：適合度の総和
    omega：ルール後件部
    w：ルールの重み
    mse：平均2乗誤差
    c：関数の中心
    b：関数の幅
    '''
    
    data = openpyxl.load_workbook('./data/'+data_filename, data_only=True)
    data_sheet = data['osaka_data_before']
    start_row = 2
    data = get_list_2d(data_sheet, start_row, total_data_num+start_row-1, 2, x_dimension+1)
    Y = get_list_2d(data_sheet, start_row, total_data_num+start_row-1, x_dimension+2, x_dimension+2)
    y = []
    fun_data = np.empty((total_data_num, func_num, x_dimension))
    
    omega = []
    total_m = []
    w = []
    
    output = openpyxl.load_workbook('./output/osaka/'+output_filename, data_only=True) 
    m = get_list_2d_T(output['m'], 1, total_dimension, 1, total_data_num) 

    for rows in output['total_m']['A1:FJ1']:
        for cells in rows:
            total_m.append(cells.value)
    
    for rows in output['omega']['A1:A729']:
        for cells in rows:
            omega.append(cells.value)
            
    for rows in output['w']['A1:A729']:
        for cells in rows:
            w.append(cells.value)

    #w = get_list_2d_T(output['w'], 1, total_dimension, 1, 1)
    mse = np.zeros(theata)
    
    #中心と幅の初期値の設定
    c = [0, 0.5, 1] 
    b = [1, 0.5, 1] #変えるべき
    
    #データのセット
    set_data(y, Y, data, b, c, omega)
    print("データセット完了")
    
    #参照問題でここで正規化
    #保存して、読み込みでもいいかも
    mm = MinMaxScaler(feature_range=(0, 1))
    data = mm.fit_transform(data)
    print("正規化完了")
    
    threshold = np.arange(0,max(w),max(w)/theata)
    
    #pprint.pprint(w)
    make_output(total_m, m, y, fun_data, data, omega, c, b, w, mse, threshold)
    #pprint.pprint(y)
    wc = openpyxl.load_workbook('./output/osaka/'+save_filename, data_only=True)
    inversed = np.multiply(y, np.max(Y)-np.min(Y))+np.min(Y)
    write_list_2d(wc["fix"],[['理想値','出力値']], 1, 1)
    write_list_2d(wc["fix"],inversed, 2, 1)
    wc.save('./output/osaka/'+save_filename)
    
if __name__ == '__main__':
    
    main()

データセット完了
正規化完了
学習開始
0 0.0
1 0.4938109572164745
2 0.987621914432949
3 1.4814328716494236
Error total_m is 0
Error total_m is 0
Error total_m is 0
Error total_m is 0
4 1.975243828865898
Error total_m is 0
Error total_m is 0
Error total_m is 0
Error total_m is 0
5 2.4690547860823724
Error total_m is 0
Error total_m is 0
Error total_m is 0
Error total_m is 0
6 2.962865743298847
Error total_m is 0
Error total_m is 0
Error total_m is 0
Error total_m is 0
7 3.4566767005153216
Error total_m is 0
Error total_m is 0
Error total_m is 0
Error total_m is 0
8 3.950487657731796
Error total_m is 0
Error total_m is 0
Error total_m is 0
Error total_m is 0
9 4.444298614948271
Error total_m is 0
Error total_m is 0
Error total_m is 0
Error total_m is 0
Already exist. Save complited 
学習完了
